In [35]:
import os
import csv
from tqdm import tqdm
import tweepy
import pandas as pd
import time
from datetime import datetime
import nltk


CONSUMER_KEY='DfMnPVzPIRS1jyhwEEtIWVNYF'
CONSUMER_SECRET='3YVosIbf8GBGahVCMfjHEVNRBUGb00CI5N4OVkzelwmM5jXVbn'
ACCESS_KEY='1640818273-xsgeYxwxsz5T1FCrE1OhJnR6lvURfai1ija02DW'
ACCESS_SECRET='DNISI77zYk4IN4znu29VcjkPLRKgSWsnXua1i3XjtPlaE'

In [37]:
auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY,ACCESS_SECRET)

In [68]:
#DATABASE

def get_tweets(feature_keywords, num_tweets=100, f_text="extended", retweets="retweeted_status"):

    start_date = "202111010900" 
    end_date = "202112010900"  
    api = tweepy.API(auth, wait_on_rate_limit=True)

    fetch_feature_tweets = api.search_full_archive(label="darin", query=feature_keywords, fromDate=start_date, 
                                                   toDate=end_date, maxResults=num_tweets) 

    all_feat_tweets = []
    for ind, tweet in enumerate(tqdm(fetch_feature_tweets)):
        status = api.get_status(tweet.id, tweet_mode=f_text)
        if hasattr(status, retweets):
            full_text = status.retweeted_status.full_text.encode("utf-8").lower()
            feat_tweets = [tweet.user.screen_name, full_text, tweet.retweeted_status.user.screen_name,
                           tweet.entities['hashtags'], tweet.created_at, tweet.user.followers_count,
                           tweet.user.friends_count, tweet.lang]
            if tweet.lang == 'en':
                all_feat_tweets.append(feat_tweets)
        else:
            full_text = status.full_text.encode("utf-8").lower()
            feat_tweets = [tweet.user.screen_name, full_text, "no retweet username", tweet.entities['hashtags'],
                           tweet.created_at, tweet.user.followers_count, tweet.user.friends_count,tweet.lang]
            if tweet.lang == 'en':
                all_feat_tweets.append(feat_tweets)


    path = os.getcwd()
    filename = path + '/' + '2021vaccine.csv'

    with open(filename, 'a') as firstcsv:
        writer = csv.writer(firstcsv)
        writer.writerow(["username", "tweet", "username_of_retweeter", "hashtag", "tweet created",
                         "number of followers", "# following", "language"])
        writer.writerows(all_feat_tweets)


if __name__ == "__main__":
    f_keywords_list = ['Pfizer', 'Pfizervaccine','Pfizer vaccine']
    for f_keywords in f_keywords_list: 
        #PfizerBioNTech OR Pfizer OR Pfizervaccine OR PfizerProud OR PfizerBioN OR Pfizer vaccine
        get_tweets(f_keywords)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.15it/s]
